In [ ]:
import os
from typing import Tuple

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import pyproj
from pystac_client import Client
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import shapely
from shapely.geometry import LinearRing, Polygon
from shapely.ops import transform


parks = [
    (
        "Tiergarten",
        "POLYGON ((13.351501 52.513962, 13.360061 52.514732, 13.370124 52.511389, 13.363086 52.510175, 13.352037 52.510018, 13.351501 52.513962))",
    ),
    (
        "Humboldthain",
        "POLYGON ((13.381429 52.544103, 13.383102 52.542851, 13.389539 52.545043, 13.387608 52.547914, 13.384089 52.547679, 13.381858 52.5464, 13.383102 52.545461, 13.381429 52.544103))",
    ),
    (
        "VolksparkFriedrichshain",
        "POLYGON ((13.427434 52.527997, 13.437046 52.529068, 13.438548 52.528624, 13.44211 52.53053, 13.445027 52.528076, 13.440951 52.526979, 13.439192 52.527841, 13.435158 52.526483, 13.437819 52.524055, 13.434128 52.523663, 13.427434 52.527997))",
    ),
    (
        "TreptowerPark",
        "POLYGON ((13.460611 52.490908, 13.462799 52.493155, 13.472153 52.491143, 13.478933 52.484662, 13.473441 52.482101, 13.460611 52.490908))",
    ),
    (
        "TempelhoferFeld",
        "POLYGON ((13.395079 52.479696, 13.405463 52.479226, 13.415418 52.475671, 13.415847 52.470443, 13.405034 52.467096, 13.394993 52.469606, 13.390874 52.475043, 13.395079 52.479696))",
    ),
    (
        "Hasenheide",
        "POLYGON ((13.41027 52.487746, 13.409412 52.484714, 13.410828 52.484113, 13.410528 52.482414, 13.418809 52.480585, 13.420182 52.481108, 13.420054 52.486857, 13.41027 52.487746))",
    ),
    (
        "Grunewald",
        "POLYGON ((13.189399 52.434874, 13.250846 52.483617, 13.208623 52.497414, 13.193518 52.489052, 13.197294 52.483198, 13.193518 52.477344, 13.195235 52.472534, 13.200727 52.471279, 13.196608 52.46354, 13.191459 52.462285, 13.190772 52.454963, 13.191115 52.448059, 13.18116 52.44764, 13.181504 52.439269, 13.189399 52.434874))",
    ),
    (
        "GaertenDerWelt",
        "POLYGON ((13.571628 52.540188, 13.579952 52.53917, 13.580467 52.537317, 13.576863 52.536351, 13.574932 52.534655, 13.568753 52.53502, 13.571628 52.540188))",
    ),
    (
        "ParkAmGleisdreieck",
        "POLYGON ((13.374437 52.496512, 13.377054 52.496146, 13.376282 52.495193, 13.377569 52.494945, 13.37905 52.496891, 13.378664 52.49403, 13.376282 52.49339, 13.375789 52.49207, 13.3736 52.492345, 13.373514 52.494579, 13.374437 52.496512))",
    ),
]

In [ ]:
# search parameters
DATE_RANGE = "2022-01-01/2022-12-31"
# if an image has less than <MAX_CLOUD_COVER_SEARCH> % of clouds
# overall, we add it to our search and check the precise 
# cloud cover over our area of interest in the next step
MAX_CLOUD_COVER_SEARCH = 80


def execute_sentinel2_query(catalog: Client, bounding_box: tuple, 
                            date_range: str, max_cloud_cover: int
                            ) -> gpd.GeoDataFrame:
    """find sentinel 2 TCI images with max_cloud_cover 
    in date_range that overlap bounding_box.
    Returns DF with metadata of relevant images"""
    query = catalog.search(
        collections=["sentinel-2-l2a"], datetime=date_range, 
        limit=200, bbox=bounding_box,
        query={"eo:cloud_cover":{"lt":max_cloud_cover}},
    )
    stac_json = query.item_collection_as_dict()
    return gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")


catalog = Client.open("https://earth-search.aws.element84.com/v1")

image_metadata_df = []
for (area_name, area_wkt) in parks:
    area_geom = shapely.from_wkt(area_wkt)
    area_bbox = area_geom.bounds
    images_over_area = execute_sentinel2_query(catalog, area_bbox, 
                                  DATE_RANGE, MAX_CLOUD_COVER_SEARCH)
    images_over_area["area_name"] = area_name
    images_over_area["area_geom"] = area_geom
    image_metadata_df.append(images_over_area)    
     

image_metadata_df = pd.concat(image_metadata_df)
print(f"Total number of images: {len(image_metadata_df)}")
print(f"Median number of images per area:", 
      image_metadata_df.groupby("area_name").size().median(), "\n")


image_metadata_df[["area_name", "s2:granule_id", "datetime", 
     "eo:cloud_cover", "earthsearch:s3_path"]].sample(3).head()


In [ ]:
# if the area of interest has more than MAX_CLOUD_COVER_PRECISE % cloud
# cover, we do not download it
MAX_CLOUD_COVER_PRECISE = 2

def project_shape(shape: Polygon, from_proj_epsg: int, 
                  to_proj_epsg: int) -> Polygon:
    """Projects a shapely Polygon to a new CRS"""
    from_crs = pyproj.CRS('EPSG:'+str(from_proj_epsg))
    to_crs = pyproj.CRS('EPSG:'+str(to_proj_epsg))
    project = pyproj.Transformer.from_crs(
        from_crs, to_crs, always_xy=True).transform
    return transform(project, shape)


def s3path_to_tif_download_link(s3path:str, image_type: str):
    """ Convert base s3 path for sentinel-2 image product into
    link to .tif file for true-color-image (TCI) or scene 
    classification (SCL). <image_type> must be either TCI or SCL"""
    s3url_base = s3path.removeprefix("s3://sentinel-cogs")
    s3url_base = "https://sentinel-cogs.s3.us-west-2.amazonaws.com" \
        + s3url_base
    if image_type == "SCL":
        return s3url_base + "/SCL.tif"
    elif image_type == "TCI":
        return s3url_base + "/TCI.tif"
    else:
        raise ValueError(
            f"Image type must be 'TCI' or 'SCL', Got {image_type} instead")


def get_precise_cloud_percentage(s3_scl_url: str, 
                               shape: Polygon, 
                               ) -> float:
    """ Get percentage of cloud or cloud shadow
    pixels in the Scene Classification at <s3_scl_url> 
    cropped to <shape> """
    with rasterio.open(s3_scl_url) as src:
        out_image, _ = mask(src, [shape], crop=True)
    # these values represent errors, clouds, or cloud shadows
    # see https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm-overview#classification-mask-generation
    bad_vals = [1, 3, 8, 9, 10]
    bad_vals_count = np.bincount(out_image.flatten(),
                                 minlength=11)[bad_vals].sum()
    bad_vals_perc = bad_vals_count/np.count_nonzero(out_image)*100
    return bad_vals_perc


def check_clouds_crop_save(s3path: str, raster_proj_epsg: int, 
                      save_folder: int, shape: Polygon, 
                      max_cloud_cover_perc: int,
                      shape_proj_epsg=4326,
                      ) -> Tuple[str, np.ndarray]:
    """ Download part of raster image at <s3path> overlapping <shape>,
    checks whether overlapping area has less than <max_cloud_cover_perc> clouds,
    and if so saves the cropped raster to <save_folder>. 
    Returns path of saved raster"""

    # 1. project shape into same CRS as raster image
    shape_projected = project_shape(shape, shape_proj_epsg, 
                                      raster_proj_epsg)
    
    # 2. check if the image masked to the shape contains clouds
    # by analyzing scene classification file (SCL)
    s3url_scl = s3path_to_tif_download_link(s3path, "SCL")
    try:
        cloud_perc = get_precise_cloud_percentage(s3url_scl, shape_projected)
        if (cloud_perc > max_cloud_cover_perc):
            print(f"{s3path}: Too many clouds over area of interest ({cloud_perc:.2f}%)")
            return None, None
    # rasterio throws ValueError if image does not intersect with shape
    # this happens if bounding box used in query overlaps with image, 
    # but not the actual shape
    except ValueError:
        print(f"{s3path}: Area of interest and image do not overlap")
        return None, None
   
    # 3. download true-color image (TCI) overlapping shape into memory
    with rasterio.open(s3path_to_tif_download_link(s3path, "TCI")) as src:
        out_image, out_transform = mask(src, [shape_projected], 
                                        crop=True)
        out_meta = src.meta
    
    # 4. write cropped image to file
    os.makedirs(save_folder, exist_ok=True)
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
    out_path = save_folder + s3path.split("/")[-1] + "-TCI_cropped.tif"
    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(out_image)
    print(f"{s3path}: Wrote image to {out_path}")

    return out_path


image_metadata_df = image_metadata_df.head(10)
# save the cropped images into folders grouped by areas of interest
name_to_folder = lambda name: "./data/" + name + "/"
image_metadata_df["file"] = None
image_metadata_df["file"] = image_metadata_df.apply(
    lambda row: check_clouds_crop_save(
        s3path=row["earthsearch:s3_path"], 
        raster_proj_epsg=row["proj:epsg"], 
        shape=row["area_geom"], 
        save_folder=name_to_folder(row["area_name"]),
        max_cloud_cover_perc=MAX_CLOUD_COVER_PRECISE),
    axis=1)

downloaded_images = image_metadata_df[image_metadata_df.file.notna()]
downloaded_images.to_csv("./data/images_overview.csv", index=False)

downloaded_images[["area_name", "datetime"]].sample(3).head()


## Plot clouds 

In [ ]:
area_wkt = parks[0][1]
area_geom = shapely.from_wkt(area_wkt)
area_bbox = area_geom.bounds
date_range = "2022-01-01/2022-12-31"
df = execute_sentinel2_query(catalog, area_bbox, date_range, max_cloud_cover=100)
print(f"Number of imgs: {len(df)}")
df["tif_link_clouds"] = df.apply(lambda r: s3path_to_tif_download_link(r["earthsearch:s3_path"], "SCL"), axis=1)
df["tif_link_tci"] = df.apply(lambda r: s3path_to_tif_download_link(r["earthsearch:s3_path"], "TCI"), axis=1)
df["precise_cloud_cover"] = df.apply(lambda r: get_precise_cloud_percentage(r["tif_link_clouds"], project_shape(area_geom, 4326, r["proj:epsg"])), axis=1)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

fig, ax = plt.subplots()
ax.scatter(x=df["eo:cloud_cover"], y=df["precise_cloud_cover"])
ax.set_xlabel("Cloud cover [%] in whole image")
ax.set_ylabel("Cloud cover [%] in area of interest")
file_path = "./imgs/cloud_cover_scatter.png"
plt.savefig(file_path, dpi=300)

In [ ]:
from shapely import buffer, box
from rasterio.plot import show

plt.rcParams.update({'font.size': 18})

# plot image that has high global and low local cloud coevr
example1 = df[df["precise_cloud_cover"]==0].sort_values("eo:cloud_cover", ascending=False).iloc[6]

fig, ax = plt.subplots(figsize=(20,10), ncols=2)

# on the left, we plot the full image
shape_to_plot = project_shape(area_geom, 4326, example1["proj:epsg"])
shape_gdf = gpd.GeoDataFrame({"aoi": [shape_to_plot]}, geometry="aoi", crs=example1["proj:epsg"])
with rasterio.open(example1.tif_link_tci) as src:
    show(src, ax=ax[0])
shape_gdf.plot(ax=ax[0], color="red")
ax[0].set_title(f"Full image")

# on the right, we plot a zoomed exceprt around the area of interest
buffered_shape = box(*buffer(shape_to_plot, 2000).bounds)
with rasterio.open(example1.tif_link_tci) as src:
    out_image, out_transform = mask(src, [buffered_shape], 
                                    crop=True)
    show(out_image, ax=ax[1], transform=out_transform)

shape_gdf.plot(ax=ax[1], edgecolor="red", facecolor="none", linewidth=2)
ax[1].set_title(f"Zoomed to area of interest")

plt.suptitle(f"Cloud cover in whole image: {example1['eo:cloud_cover']:.0f}%\n Cloud cover in area of interest (red polygon): {example1.precise_cloud_cover:.0f} %", y=0.95) 

file_path = "./imgs/cloud_cover_example1.png"
plt.savefig(file_path, dpi=300)
plt.show()




In [ ]:

# plot image that has low global and high local cloud coevr
example2 = df[df["precise_cloud_cover"]==100].sort_values("eo:cloud_cover", ascending=True).iloc[12]

fig, ax = plt.subplots(figsize=(20,10), ncols=2)

# on the left, we plot the full image
shape_to_plot = project_shape(area_geom, 4326, example2["proj:epsg"])
shape_gdf = gpd.GeoDataFrame({"aoi": [shape_to_plot]}, geometry="aoi", crs=example2["proj:epsg"])
with rasterio.open(example2.tif_link_tci) as src:
    show(src, ax=ax[0])
shape_gdf.plot(ax=ax[0], color="red")
ax[0].set_title(f"Full image")

# on the right, we plot a zoomed exceprt around the area of interest
buffered_shape = box(*buffer(shape_to_plot, 2000).bounds)
with rasterio.open(example2.tif_link_tci) as src:
    out_image, out_transform = mask(src, [buffered_shape], 
                                    crop=True)
    show(out_image, ax=ax[1], transform=out_transform)

shape_gdf.plot(ax=ax[1], edgecolor="red", facecolor="none", linewidth=2)
ax[1].set_title(f"Zoomed to area of interest")

plt.suptitle(f"Cloud cover in whole image: {example2['eo:cloud_cover']:.0f}%\n Cloud cover in area of interest (red polygon): {example2.precise_cloud_cover:.0f} %", y=0.95) 

file_path = "./imgs/cloud_cover_example2.png"
plt.savefig(file_path, dpi=300)
plt.show()


